In [1]:
# Import packages

import json, string

import folium
import requests
import geocoder
import pandas as pd

from folium import Map, Marker, GeoJson, LayerControl
from ediblepickle import checkpoint

%matplotlib inline

In [2]:
# Read API keys from file

with open("secrets/.wmata") as fin:
    wmata_key = fin.read().strip()
with open("secrets/.walkscore") as fin:
    walkscore_key = fin.read().strip()

# Folium: A Leaflet Wrapper for Python

What can Folium and by extension, Leaflet can do to help you explore the structure of cities?

1. [Live bus tracking](#Bus-tracker)
1. [Neighborhood choropleths](#Neighborhood-choropleths)
1. [Walk Score&reg; along bus routes](#Walk-Score)

[Leaflet](http://leafletjs.com) is currently one of the most popular JavaScript libraries for mapping.
* Interactive through web browser
* Flexibility of JavaScript
* Mobile-friendly
* Great API
* Open source

[Python](https://www.python.org) is a great language for the web. Using the same language, you can
* Get data from APIs or scrape the web with libraries like [requests](docs.python-requests.org) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/)
* Process and analyze data with [NumPy, SciPy](https://docs.scipy.org), and [Pandas](http://pandas.pydata.org)
* Model relationships with state-of-the-art machine learning library [Scikit-learn](http://scikit-learn.org)
* Build a web framework for your project with [Flask](http://flask.pocoo.org) and [Jinja2](http://jinja.pocoo.org)
* Host your application with anything from [Heroku](http://www.heroku.com) to [Google App Engine](https://developers.google.com/appengine).

Python's continuing utility is due to its *ecosystem*, not its syntax or speed. For example, according to Wikipedia, TensorFlow was the fastest growing deep learning framework in fall 2016. The second fastest was a high-level Python library called Keras which can seamlessly plug into TensorFlow's architecture.

There's a similar relationship between Leaflet.js and the Python package Folium.

## Folium

Folium uses the Leaflet API to allow users to write Python code to generate and manipulate interactive JavaScript maps. This also allows for drawing those maps in Jupyter notebooks.

### Interactivity

You can drag, zoom, click, and hover. More generally you can provide **input** and get **output**, even if that output requires backend calculations. JavaScript was designed to ferry information between the frontend and the backend seamlessly.

* More seamless than redrawing maps
* Answer questions in real time
* Simultaneous exploration and explanation

All this makes for a good tool.

### Documentation

* [GitHub](https://github.com/python-visualization/folium)
* [Docs and API](http://python-visualization.github.io/folium/index.html)
* [Examples](https://github.com/python-visualization/folium/tree/master/examples)

## Bus tracker

The WMATA API is free to use. Learn more [here](https://developer.wmata.com).

* Investigate real time positions compared to scheduled locations to say something about actual vs. ideal transit
* Look at how connections between regions vary throughout the day

Interactivity:
* Input: location of interest, search radius
* Output: locations and information of buses in the area

### Geocoder

An alternative to calling eg. the OpenStreetMap API, [geocoder](http://geocoder.readthedocs.io) provides a wrapper around a number of popular geocoding services.

In [3]:
location = 'Union Station, Washington, DC'
loc = geocoder.osm(location)

In [4]:
loc.json

{'accuracy': 0.31100000000000005,
 'address': 'Union Station, Union Station Drive Northeast, NoMa, Near Northeast, Washington, D.C., 20549, USA',
 'bbox': {'northeast': [38.902766, -77.0024143],
  'southwest': [38.892766, -77.0124143]},
 'city': 'Washington',
 'confidence': 7,
 'country': 'USA',
 'country_code': 'us',
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/transport_train_station2.p.20.png',
 'importance': 0.31100000000000005,
 'lat': 38.897766,
 'lng': -77.0074143,
 'neighborhood': 'NoMa',
 'ok': True,
 'osm_id': '738189330',
 'osm_type': 'node',
 'place_id': '7131410',
 'place_rank': '30',
 'postal': '20549',
 'quality': 'station',
 'raw': {'place_id': '7131410',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': '738189330',
  'boundingbox': ['38.892766', '38.902766', '-77.0124143', '-77.0024143'],
  'lat': '38.897766',
  'lon': '-77.0074143',
  'display_name': 'Union Station, Union Station 

In [5]:
# Maps are hierarchical objects
latlng = [loc.lat, loc.lng]

bus_map = Map(location=latlng,
              zoom_start=15)
bus_map.add_child(Marker(location=latlng, popup=loc.address, icon = folium.Icon(color = 'blue')))
# bus_map.add_child(GeoJson(loc.geojson))
bus_map

In [6]:
# Saving maps

# bus_map.save('bus_map.html')

In [7]:
# Set general WMATA parameters

session = requests.Session()
session.mount('https://api.wmata.com', requests.adapters.HTTPAdapter(max_retries = 2))

headers = {'api_key': wmata_key}

radius = '1000'

### Checkpointing

Caching and checkpointing is crucial for dealing with APIs sustainably and respectfully. You should never hit an endpoint twice for the same data.

[Edible Pickle](https://pypi.python.org/pypi/ediblepickle) is a checkpointing tool that allows you to save the expensive results of a function so that it need not be run again if that result is already present.

In the following cell, setting `refresh = True` will make the function get current data instead of relying on the cache.

In [8]:
# Call API for bus locations

bus_endpoint = 'https://api.wmata.com/Bus.svc/json/jBusPositions'

@checkpoint(key = string.Template('{0}x{1}_radius{2}.buslist'), work_dir = 'cache/', refresh = False)
def get_buses(lat, lon, radius):
    """
    All values passed as strings and radius in meters
    """

    params = {
              # 'RouteID': 'B12',
              'Lat': lat,
              'Lon': lon,
              'Radius': radius
             }

    response = session.get(bus_endpoint, params = params, headers = headers)
    if not response.status_code == 200:
        raise ValueError("Response status not 200")
    else:
        return response.json()['BusPositions']

In [9]:
bus_list = get_buses(loc.lat, loc.lng, radius)
# buses_in_the_area = len(bus_list)

In [10]:
# example response element

bus_list[0]

{'VehicleID': '2960',
 'Lat': 38.897106,
 'Lon': -77.008423,
 'Deviation': -5.0,
 'DateTime': '2018-11-17T11:01:02',
 'TripID': '863927070',
 'RouteID': '96',
 'DirectionNum': 0,
 'DirectionText': 'EAST',
 'TripHeadsign': 'CAPITOL HEIGHTS STATION',
 'TripStartTime': '2018-11-17T10:14:00',
 'TripEndTime': '2018-11-17T11:36:00'}

In [11]:
for bus in bus_list:
    folium.features.RegularPolygonMarker(location = [bus['Lat'], bus['Lon']],
                                         popup = 'Route %s to %s' % (bus['RouteID'], bus['TripHeadsign']),
                                         number_of_sides = 3,
                                         radius = 10,
                                         weight = 1,
                                         fill_opacity = 0.8).add_to(bus_map)
bus_map

### Exercises

* Use the WMATA bus routes endpoint to identify a bus' next stop, and use math to rotate its triangle to the right direction
* Overlay trains in the area on the map

## Neighborhood choropleths

* Visualize metrics of interest that have different values for each region
* Overlay metrics to perform an "and" query

Interactivity:
* Input: neighborhood shape information, a value for each neighborhood to plot
* Output: neighborhood regions highlighted on map, colored by value

### GeoJSON

A file format that combines geographical data with associated JSON attributes. You can find or create these datasets in a variety of ways. This particular dataset comes from [this GitHub repository](https://github.com/benbalter/dc-maps/tree/master/maps).

In [12]:
with open('geojson/neighborhood-composition.geojson') as fin:
    gjdata = json.load(fin)
nhoods = gjdata['features']
nhoods[-1]

{'type': 'Feature',
 'properties': {'OBJECTID': 192,
  'AREA_': 4867200.5,
  'PERIMETER': 10776.585,
  'DC_CEN_TRA': 193,
  'DC_CEN_T_1': 192,
  'POP90': 709,
  'HUNITS90': 4,
  'TRACT': 73.8,
  'POVRATE': 49.2,
  'POVGROUP': 2,
  'NHOOD_COMP': 'NA',
  'SHAPE_Length': 10776.580978551374,
  'SHAPE_Area': 4867200.292597003},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-77.01742872863446, 38.82391845630681],
    [-77.01720057631805, 38.82392242686579],
    [-77.01630051535132, 38.823820787328664],
    [-77.01585749400452, 38.82374744731969],
    [-77.01260055920578, 38.823222461180535],
    [-77.01210051797308, 38.82212271420519],
    [-77.01160402306138, 38.82283865030645],
    [-77.01100879042542, 38.82040248211819],
    [-77.01096696298997, 38.81985359934235],
    [-77.01083442849452, 38.81913504120183],
    [-77.01060059084837, 38.81712068475211],
    [-77.01110030991468, 38.816521925349406],
    [-77.01110061995615, 38.81622014104926],
    [-77.00900063170232, 38.81622024989

### Colormaps

Check out [ColorBrewer](http://colorbrewer2.org) for advice about coloring for cartography.

In [13]:
# Create Pandas DataFrame

nhdata = pd.DataFrame([nhood['properties'] for nhood in nhoods], columns = sorted(nhoods[0]['properties'].keys()))
nhdata.describe()

,AREA_,DC_CEN_TRA,DC_CEN_T_1,HUNITS90,OBJECTID,PERIMETER,POP90,POVGROUP,POVRATE,SHAPE_Area,SHAPE_Length,TRACT
count,1.920000e+02,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,1.920000e+02,192.000000,192.000000
mean,9.256678e+05,97.500000,96.500000,1450.463542,96.500000,4134.226021,3160.937500,0.973958,17.318021,9.242352e+05,4136.915084,58.466667
std,1.245642e+06,55.569776,55.569776,781.478079,55.569776,2505.293802,1444.572962,0.840312,13.166314,1.242515e+06,2517.643312,30.633035
min,1.625355e+05,2.000000,1.000000,0.000000,1.000000,1675.831000,0.000000,0.000000,0.000000,1.625355e+05,1675.830873,1.000000
25%,3.529960e+05,49.750000,48.750000,1035.500000,48.750000,2711.860000,2253.500000,0.000000,7.417500,3.481418e+05,2711.859936,29.750000
50%,5.473621e+05,97.500000,96.500000,1350.500000,96.500000,3455.979500,3079.000000,1.000000,13.575000,5.510787e+05,3456.624765,64.600000
75%,1.016516e+06,145.250000,144.250000,1888.000000,144.250000,4756.924500,4182.250000,2.000000,24.025000,1.015430e+06,4765.208261,84.425000
max,1.142948e+07,193.000000,192.000000,4555.000000,192.000000,25651.359000,7767.000000,2.000000,79.630000,1.144449e+07,25611.404561,99.700000


In [14]:
# Set up colormaps to represent the range of values

from branca.colormap import linear

popcolors = linear.GnBu_06.scale(
    nhdata['POP90'].min(),
    nhdata['POP90'].max() / 1.5)

povcolors = linear.PuRd_06.scale(
    nhdata['POVRATE'].min(),
    nhdata['POVRATE'].max() / 2)

print(popcolors(1000))

povcolors

#ceecc6


In [15]:
nh_map = Map(location = latlng,
             zoom_start = 13,
             tiles = 'Stamen Toner')

In [16]:
# Adds a caption to the map that shows the color scale

popcolors.caption = 'Population Scale'
popcolors.add_to(nh_map)

In [17]:
GeoJson(gjdata,
        name = 'population',
        style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillColor': popcolors(feature['properties']['POP90'])
            }
        ).add_to(nh_map)

In [18]:
GeoJson(gjdata,
        name = 'poverty rate',
        style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillColor': povcolors(feature['properties']['POVRATE'])
            }
        ).add_to(nh_map)

In [19]:
LayerControl().add_to(nh_map)

In [20]:
# Colormaps can be changed on the fly

nh_map

### More succinct choropleths

This [example notebook](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb) goes through some other techniques for creating choropleths with additional functionality all within one choropleth method.

### Exercises

* ~~Redo the choropleth with population density instead of raw counts~~
* Come up with a standard way of setting colormap thresholds for different variable distributions
* Get a shapefile from the DC OpenData API and [convert it to GeoJSON](https://ben.balter.com/2013/06/26/how-to-convert-shapefiles-to-geojson-for-use-on-github/) using [ogr2ogr](https://ogre.adc4gis.com) or a similar tool

In [21]:
# Using Pandas to create derived variables

nhdata['density'] = nhdata['POP90'] / nhdata['AREA_']
nhdata.tail()

,AREA_,DC_CEN_TRA,DC_CEN_T_1,HUNITS90,NHOOD_COMP,OBJECTID,PERIMETER,POP90,POVGROUP,POVRATE,SHAPE_Area,SHAPE_Length,TRACT,density
187,203108.859,189,188,884,E,188,2123.158,2209,2,34.12,2.031088e+05,2123.158869,98.2,0.010876
188,342408.281,190,189,643,E,189,2694.009,1353,2,22.80,3.424085e+05,2694.011243,98.5,0.003951
189,548306.625,191,190,2621,E,190,3689.665,5921,2,32.33,5.483066e+05,3689.666505,98.6,0.010799
190,529116.313,192,191,1443,E,191,3206.134,2380,1,18.93,5.291161e+05,3206.132327,98.8,0.004498
191,4867200.500,193,192,4,NA,192,10776.585,709,2,49.20,4.867200e+06,10776.580979,73.8,0.000146


In [22]:
# update geojson
for nhood in gjdata['features']:
    nhood['properties']['density'] = nhdata.loc[nhood['properties']['OBJECTID'] - 1].density

In [23]:
nh_map = Map(location = latlng,
             zoom_start = 13,
             tiles = 'Stamen Toner')

In [24]:
densitycolors = linear.GnBu_06.scale(
    nhdata['density'].min(),
    nhdata['density'].max() / 2)

densitycolors.caption = 'Population Density Scale'
densitycolors.add_to(nh_map)

In [25]:
GeoJson(gjdata,
        name = 'density',
        style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillColor': densitycolors(feature['properties']['density'])
            }
        ).add_to(nh_map)

In [26]:
GeoJson(gjdata,
        name = 'poverty rate',
        style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillColor': povcolors(feature['properties']['POVRATE'])
            }
        ).add_to(nh_map)

In [27]:
LayerControl().add_to(nh_map)
nh_map

## Walk Score

A measure of how dependent an address is on having a car. For example, areas that require cars are more expensive than they seem to live in. An [API](https://www.walkscore.com/professional/api.php) is available.

<a href = "https://www.walkscore.com/how-it-works/"><img src = 'https://cdn.walk.sc/images/api-logo.gif' style = "float: left; margin: 0px 10px 0px 0px" /></a>
[84](https://www.walkscore.com/how-it-works/)
<a href = "https://www.redfin.com/how-walk-score-works"><img src = 'https://cdn.walk.sc/images/api-more-info.gif' style = "float: right; margin: 0px 810px 0px 0px" /></a>

* Investigate how transit routes lie on top of underlying people - what's between the bus stops?
* Is the purpose of a bus route to increase "walkability" for populations?

Interactivity:
* Input: bus route
* Output: line along the shape of the route, colored by score

In [28]:
# Get Metrobus route data from WMATA

route_endpoint = 'https://api.wmata.com/Bus.svc/json/jRouteDetails'
date = '2018-11-01'  # This needs to be changed periodically

def get_route_shape(route, date):  # eg. 'L2', 'YYYY-MM-DD'
    params = {'RouteID': route}
    if date:
        params['Date'] = date
    response = session.get(route_endpoint, params = params, headers = headers)
    if response.status_code != 200:
        raise ValueError("Error: Response status not 200")
    else:
        return response.json()['Direction0']['Shape']

# for stop in route_shape:
#     Marker(location=[stop['Lat'], stop['Lon']], popup=str(stop['SeqNum'])).add_to(m)

In [29]:
# Get Walkscore data - 500 ft grid resolution

"""
http://api.walkscore.com/score?format=json&
address=1119%8th%20Avenue%20Seattle%20WA%2098101&lat=47.6085&
lon=-122.3295&transit=1&bike=1&wsapikey=<YOUR-WSAPIKEY>
"""

walkscore_endpoint = 'http://api.walkscore.com/score'

def get_walkscore(pin):
    params = {'format': 'json',
              'wsapikey': walkscore_key,
              'lat': pin[0],
              'lon': pin[1],
              'transit': '1',
              'bike': '1',
              'address': geocoder.osm(pin, method='reverse').address}
    response = requests.get(walkscore_endpoint, params = params)
    if response.status_code != 200:
        return None
    else:
        return response.json()

In [30]:
@checkpoint(key = string.Template('{0}_scores_{1}.panda'), work_dir = 'cache/', refresh = False)
def get_route_scores(route, date):
    shape = get_route_shape(route, date)
    pins = [(pin['Lat'], pin['Lon']) for pin in shape]
    
    walk_scores = []
    transit_scores = []
    bike_scores = []
    for pin in pins:
        score_json = get_walkscore(pin)
        if not score_json:
            walk_scores.append(-1)
            transit_scores.append(-1)
            bike_scores.append(-1)
            continue

        walk_scores.append(score_json.get('walkscore', -1))
        transit_scores.append(score_json.get('transit', {}).get('score', -1))
        bike_scores.append(score_json.get('bike', {}).get('score', -1))
    df = pd.DataFrame({'pin': pins,
                       'walk_score': walk_scores,
                       'transit_score': transit_scores,
                       'bike_score': bike_scores})
    df = df[['pin', 'walk_score', 'bike_score', 'transit_score']]
    return df

In [31]:
# Example response from Walk Score API

test = get_walkscore(latlng)

In [32]:
test

{'status': 1,
 'walkscore': 96,
 'description': "Walker's Paradise",
 'updated': '2018-09-07 15:42:23.727460',
 'logo_url': 'https://cdn.walk.sc/images/api-logo.png',
 'more_info_icon': 'https://cdn.walk.sc/images/api-more-info.gif',
 'more_info_link': 'https://www.redfin.com/how-walk-score-works',
 'ws_link': 'https://www.walkscore.com/score/H.and.M-Union-Station-Drive-Northeast-NoMa-Near-Northeast-Washington-D.C.-20549-USA/lat=38.897766/lng=-77.0074143/?utm_source=github.com&utm_medium=ws_api&utm_campaign=ws_api',
 'help_link': 'https://www.redfin.com/how-walk-score-works',
 'snapped_lat': 38.898,
 'snapped_lon': -77.007,
 'transit': {'score': None, 'description': None, 'summary': None},
 'bike': {'score': 94, 'description': "Biker's Paradise"}}

In [33]:
# This is where the magic happens
# Maps here: https://www.wmata.com/schedules/maps/

route = 'L2'

df = get_route_scores(route, date)

In [34]:
print(df.shape)
df.head()

(42, 4)


,pin,walk_score,bike_score,transit_score
0,"(38.901615, -77.038596)",97,90,None
1,"(38.90133, -77.03852)",97,90,None
2,"(38.90135, -77.03941)",98,91,None
3,"(38.90283, -77.039389)",98,89,None
4,"(38.903556, -77.039748)",97,89,None


In [35]:
score_map = Map(location = loc.latlng, zoom_start = 12, tiles = 'Stamen Terrain')

In [36]:
color_line = folium.features.ColorLine(
    df['pin'],
    colors = df['walk_score'],
    colormap = ['y', 'orange', 'r'],
    weight = 6,
    name = u'Route %s Walk Score\u00ae' % route)
color_line.add_to(score_map)

In [37]:
# This adds the population layer back in

GeoJson(gjdata,
        name = 'population',
        style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillColor': popcolors(feature['properties']['POP90'])
            }
        ).add_to(score_map)

In [38]:
LayerControl().add_to(score_map)

In [39]:
score_map

In [40]:
# score_map.save("score_map.html")

### Exercises

* Use a for loop to add all colorlines to the map
* Overlay Walk Score fluctuations with census block or block group populations
* Cluster bus routes by "purpose" based on how much time they spend in highly walkable areas

## Call to Action

"There's a lot of energy behind open source."

Go learn, go do, go share!

&copy; Ariel M'ndange-Pfupfu 2018